When evaluating the neural network scheme in SAM, I found some strange behavior in the meridional wind field. Specifically, the Hadley and Ferrel cells seems to disappear within 1 day.

In the simulations below, I added hyper diffusion of varying strengths using some code that Matt Wyant gave me. I show three cases, each initialized with the same data.

1. The "True" NGqua data
2. A coarse resolution SAM (cSAM) simulation with hyper diffisuivity of 1e16 m^4/s.
3. A cSAM simulation with active neural network and also hyper diffusivity. the neural network is trained for many samples, and uses SOLIN and SST as an input.
4. Another neural network simulation, this time w/o SOLIN and SST as inputs.

In [ ]:
runs = {
    'NGAqua': '/Users/noah/Data/2018-05-30-NG_5120x2560x34_4km_10s_QOBS_EQX/coarse/2d/all.nc',
    'NoNN': '../../data/runs/khyp_1e16/OUT_2D/NG1_test_1.2Dbin_1.nc',
    'NN': '/Users/noah/workspace/research/uwnet/trained_models/5b/92d2bbe6060eb1f80f945e/SAM/OUT_2D/NG1_test_1.2Dbin_1.nc',
    'NN_noSOLIN': '/Users/noah/workspace/research/uwnet/trained_models/5b/92fc43e6060e6a470454ec/SAM/OUT_2D/NG1_test_1.2Dbin_1.nc'
}


time_range = slice(100, 103)


# open data
base =( xr.open_mfdataset(runs['NGAqua'])
       .sortby('time')
       .sel(time=time_range))

datasets = {}
for key, val in runs.items():
    print(f"Opening {key}")
    datasets[key] = xr.open_dataset(val).sortby('time').sel(time=base.time, method='nearest')

What does the zonally averaged meridional wind look like for these datasets?

In [ ]:
fig, axs = plt.subplots(2,2, figsize=(8,6), constrained_layout=True)

for ax, (key, val) in zip(axs.flat, datasets.items()):
    val.VSFC.mean('x').plot(x='time', ax=ax)
    ax.set_title(key)

As we can see the Ferrel cell quickly dissapears for all of the runs.

In [ ]:
fig, axs = plt.subplots(2,2, figsize=(8,6), constrained_layout=True)

for ax, (key, val) in zip(axs.flat, datasets.items()):
    val.W500.mean('x').plot(x='time', ax=ax)
    ax.set_title(key)

We see a correspondingly strange vertical velocity structure.

Here is precipitaton:

In [ ]:
fig, axs = plt.subplots(2,2, figsize=(8,6), constrained_layout=True)

for ax, (key, val) in zip(axs.flat, datasets.items()):
    try:
        val.Prec.mean('x').plot(x='time', ax=ax)
    except:
        pass
    ax.set_title(key)

In [ ]:
for key, val in datasets.items():
    try:
        val.Prec[::6].plot(col='time', col_wrap=4, vmax=100)
        plt.suptitle(key, y=1.05)
    except AttributeError:
        pass

In [ ]:
import holoviews as hv
hv.extension('bokeh')

%opts Image[colorbar=True, width=600, height=300](cmap='RdBu')

In [ ]:
%%opts Image[colorbar=True, width=600, height=300](cmap='RdBu')

plot_runs = dict(no_nn = xr.open_mfdataset(runs['NoNN']).VSFC,
ngaqua = xr.open_mfdataset(runs['NGAqua']).sortby('time').VSFC.sel(time=time_range))


def dplot(run, t):
    return hv.Image(plot_runs[run].sel(time=t, method='nearest'))

dmap = hv.DynamicMap(dplot, kdims=['run', 't'])
dmap.redim.values(run=list(plot_runs.keys()), t=plot_runs['no_nn'].time.values).layout('run').cols(1).redim.range(VSFC=(-30,30))

In [ ]:
plot_runs['ngaqua'].isel(y=50, time=0).plot()
plot_runs['no_nn'].isel(y=50, time=0).plot()

These just shows for a check that we are using the staggered data to initialize the simulations:

In [ ]:
ic = xr.open_dataset('../../data/runs/khyp_1e16/NG1/ic.nc')

ic_ng = xr.open_dataset("/Users/noah/Data/2018-05-30-NG_5120x2560x34_4km_10s_QOBS_EQX/stagger/3d/all.nc").sel(time=ic.time)

ic_ng.V.isel(z=1, ys=50).plot()
ic.V.isel(z=1, y=50).plot()

Is this initial data simply sharp or something?